In [ ]:
import pandas as pd
import random

# ---------------- ORPHANAGES ----------------
orphanage_base_names = [
    'Hope Haven', 'Sunrise Shelter', 'Little Angels', 'Starlight Home', 'New Dawn Orphanage',
    'Bright Futures', 'Happy Hearts', 'Dreamcatcher Home', 'Rainbow Shelter', 'Shining Stars',
    'Golden Wings', 'Safe Harbor', 'Silver Linings', 'Hearts of Gold', 'Gentle Hands',
    'Kindred Spirits', 'Lighthouse Home', 'Guardian Angels', 'Evergreen Shelter', 'Peaceful Haven'
]

locations = ['London', 'New York', 'Paris', 'Chennai', 'Coimbatore', 'Madurai', 'Salem', 'Tirunelveli', 'Bangalore', 'Hyderabad']
items_needed = ['Books', 'Clothes', 'Food', 'Toys', 'Stationery', 'Medicine']

orphanage_records = []
for i in range(50):
    orphanage_records.append({
        'name': f"{random.choice(orphanage_base_names)} {i+1}",
        'location': random.choice(locations),
        'items_needed': random.choice(items_needed),
        'urgency': random.randint(1,10)
    })

orphanages_df = pd.DataFrame(orphanage_records)
orphanages_df.to_csv('orphanages.csv', index=False)
print("✅ orphanages.csv created with 50 records!")

# ---------------- DONORS ----------------
donor_first_names = [
    'Eleanor', 'Sebastian', 'Victoria', 'Oliver', 'Isabella', 'Lucas', 'Charlotte', 'Henry',
    'Sophia', 'Benjamin', 'Amelia', 'James', 'Harper', 'Alexander', 'Ella', 'Noah', 'Liam',
    'Grace', 'Samuel', 'Emily', 'Jack', 'Lily', 'Daniel', 'Hannah', 'Nathan', 'Mia', 'William',
    'Scarlett', 'Ethan', 'Victoria'
]

donor_last_names = [
    'Smith', 'Johnson', 'Brown', 'Taylor', 'Anderson', 'Thomas', 'Jackson', 'White',
    'Harris', 'Martin', 'Thompson', 'Garcia', 'Martinez', 'Robinson', 'Clark'
]

donor_records = []
for i in range(50):
    donor_records.append({
        'name': f"{random.choice(donor_first_names)} {random.choice(donor_last_names)}",
        'location': random.choice(locations),
        'donation_preferences': random.choice(items_needed),
        'availability': random.choices([0,1], weights=[10,90])[0]  # mostly 1
    })

donors_df = pd.DataFrame(donor_records)
donors_df.to_csv('donors.csv', index=False)
print("✅ donors.csv created with 50 records!")

# ---------------- HISTORICAL DONATIONS ----------------
historical_records = []
for i in range(50):
    historical_records.append({
        'orphanage': orphanages_df['name'].sample(1).values[0],
        'item_provided': random.choice(items_needed),
        'month': random.randint(1,12)
    })

historical_df = pd.DataFrame(historical_records)
historical_df.to_csv('historical_donations.csv', index=False)
print("✅ historical_donations.csv created with 50 records!")

In [ ]:
# ----- FINAL ALL-IN-ONE CARECONNECT DASHBOARD (CSV-BASED) -----
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, clear_output
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

sns.set_style("whitegrid")

# ----- LOAD CSV DATA -----
orphanages = pd.read_csv("orphanages.csv")
donors = pd.read_csv("donors.csv")
historical_data = pd.read_csv("historical_donations.csv")

# ----- MACHINE LEARNING PREPARATION -----
le_orphanage = LabelEncoder()
le_item = LabelEncoder()
if not historical_data.empty:
    le_orphanage.fit(historical_data['orphanage'])
    le_item.fit(historical_data['item_provided'])
    historical_data['Orphanage_Code'] = le_orphanage.transform(historical_data['orphanage'])
    historical_data['Item_Code'] = le_item.transform(historical_data['item_provided'])
    X = historical_data[['Orphanage_Code','month']]
    y = historical_data['Item_Code']
else:
    X = pd.DataFrame()
    y = pd.Series(dtype=int)

# Train models
n_samples = len(X)
knn_neighbors = min(5, n_samples) if n_samples>0 else 1
models = {
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "KNN": KNeighborsClassifier(n_neighbors=knn_neighbors),
    "Logistic Regression": LogisticRegression(max_iter=500),
    "SVM": SVC(),
    "Gradient Boosting": GradientBoostingClassifier()
}

if not X.empty:
    for name, model in models.items():
        model.fit(X, y)

# ----- MATCHING LOGIC -----
def generate_matches():
    matches = []
    for _, orphanage in orphanages.iterrows():
        potential_donors = donors[(donors['availability']==1) &
                                  (donors['donation_preferences']==orphanage['items_needed'])]
        same_city = potential_donors[potential_donors['location']==orphanage['location']]
        other_city = potential_donors[potential_donors['location']!=orphanage['location']]
        for _, donor in same_city.iterrows():
            matches.append({'Orphanage':orphanage['name'],
                            'Donor':donor['name'],
                            'Item':orphanage['items_needed'],
                            'Urgency':orphanage['urgency'],
                            'Location_Match':'Yes'})
        for _, donor in other_city.iterrows():
            matches.append({'Orphanage':orphanage['name'],
                            'Donor':donor['name'],
                            'Item':orphanage['items_needed'],
                            'Urgency':orphanage['urgency'],
                            'Location_Match':'No'})
    df = pd.DataFrame(matches)
    return df.sort_values(by=['Urgency','Location_Match'],ascending=[False,False]).reset_index(drop=True)

matches_df = generate_matches()

# ----- DASHBOARD WIDGETS -----
output_dashboard = widgets.Output()
output_charts = widgets.Output()
output_final_dashboard = widgets.Output()
previous_matches = pd.DataFrame()

# ----- MATCHES TABLE UPDATE -----
def update_matches_table():
    global previous_matches
    with output_dashboard:
        clear_output()
        if matches_df.empty:
            display("No matches found.")
            previous_matches = matches_df.copy()
            return
        display_df = matches_df.copy()
        def highlight_rows(row):
            if previous_matches.empty:
                return ['']*len(row)
            match_cond = ((previous_matches['Orphanage']==row['Orphanage']) &
                          (previous_matches['Donor']==row['Donor']) &
                          (previous_matches['Item']==row['Item']))
            colors = []
            for idx, val in enumerate(row):
                if not any(match_cond):
                    if row['Location_Match']=='Yes':
                        colors.append('background-color: #28a745')  # bright green for location match
                    else:
                        colors.append('background-color: #98fb98')  # lighter green for match but different location
                else:
                    colors.append('')
            return colors
        styled = display_df.style.apply(highlight_rows, axis=1)
        display(styled)
        previous_matches = matches_df.copy()

def refresh_matches():
    global matches_df
    matches_df = generate_matches()
    update_matches_table()

# ----- CHARTS -----
def refresh_charts():
    plt.close('all')
    with output_charts:
        clear_output(wait=True)
        plt.figure(figsize=(7,4))
        sns.countplot(data=orphanages, x='items_needed', hue='items_needed', dodge=False, palette='Set2', legend=False)
        plt.title('Orphanage Items Needed', fontsize=14, color='#2c3e50')
        plt.xlabel('Item', fontsize=12)
        plt.ylabel('Count', fontsize=12)
        plt.show()

        plt.figure(figsize=(6,6))
        donors['location'].value_counts().plot(kind='pie', autopct='%1.1f%%', startangle=90, colors=sns.color_palette('Set3'))
        plt.ylabel('')
        plt.title('Donors by Location', fontsize=14, color='#2c3e50')
        plt.show()

# ----- FINAL DASHBOARD FUNCTION -----
def final_dashboard():
    with output_final_dashboard:
        clear_output(wait=True)
        display(widgets.HTML("<h2 style='color:#2c3e50'>HopeAI: Intelligent Donation & Orphanage Matching System</h2>"))

        # --- Interactive NLP Extraction Panel ---
        display(widgets.HTML("<h3 style='color:#2c3e50'>Interactive NLP Extraction Panel</h3>"))
        donor_msg_input = widgets.Textarea(value='', placeholder='Type donor message here...', description='Donor Msg:', layout=widgets.Layout(width='80%', height='70px'))
        orphan_msg_input = widgets.Textarea(value='', placeholder='Type orphanage request here...', description='Orphan Msg:', layout=widgets.Layout(width='80%', height='70px'))
        output_nlp = widgets.Output()
        nlp_button = widgets.Button(description="Extract Items", button_style='info')
        def extract_items(b):
            with output_nlp:
                clear_output(wait=True)
                items_list = orphanages['items_needed'].unique().tolist()
                donor_items = [item for item in items_list if item.lower() in donor_msg_input.value.lower()]
                orphan_items = [item for item in items_list if item.lower() in orphan_msg_input.value.lower()]
                print("✅ Extracted Donor Items:", donor_items if donor_items else "None")
                print("✅ Extracted Orphanage Items:", orphan_items if orphan_items else "None")
        nlp_button.on_click(extract_items)
        display(donor_msg_input, orphan_msg_input, nlp_button, output_nlp)

        # --- Interactive ML Prediction Panel ---
        display(widgets.HTML("<h3 style='color:#2c3e50'>Interactive ML Prediction Panel</h3>"))
        orphanage_test_input = widgets.Dropdown(options=orphanages['name'].tolist(), description='Orphanage:')
        month_test_input = widgets.BoundedIntText(value=1, min=1, max=12, description='Month:')
        predict_button = widgets.Button(description="Predict Next Item", button_style='success')
        output_ml = widgets.Output()
        def predict_next(b):
            with output_ml:
                clear_output(wait=True)
                orphan_name = orphanage_test_input.value
                month = month_test_input.value
                if orphan_name not in le_orphanage.classes_:
                    fallback_item = orphanages['items_needed'].mode()[0]
                    print(f"⚠ New orphanage '{orphan_name}' detected. Fallback: {fallback_item}")
                    return
                code = le_orphanage.transform([orphan_name])[0]
                test_df = pd.DataFrame({'Orphanage_Code':[code],'month':[month]})
                for name, model in models.items():
                    try:
                        pred_code = model.predict(test_df)[0]
                        pred_item = le_item.inverse_transform([pred_code])[0]
                        print(f"{name}: {pred_item}")
                    except:
                        print(f"{name}: Prediction unavailable")
        predict_button.on_click(predict_next)
        display(orphanage_test_input, month_test_input, predict_button, output_ml)

        # --- Add Orphanage / Donor Panel ---
        display(widgets.HTML("<h3 style='color:#2c3e50'>Add Orphanage / Donor Panel</h3>"))
        # Inputs
        orphan_name_input = widgets.Text(description='Name:')
        orphan_loc_input = widgets.Text(description='Location:')
        orphan_item_input = widgets.Text(description='Item Needed:')
        orphan_urgency_input = widgets.BoundedIntText(value=1, min=1, max=10, description='Urgency:')
        donor_name_input = widgets.Text(description='Name:')
        donor_loc_input = widgets.Text(description='Location:')
        donor_pref_input = widgets.Text(description='Preference:')
        donor_avail_input = widgets.BoundedIntText(value=1, min=0, max=1, description='Available:')
        output_add_panel = widgets.Output()

        add_orphan_button = widgets.Button(description="Add Orphanage", button_style='warning')
        add_donor_button = widgets.Button(description="Add Donor", button_style='warning')

        def add_orphanage(b):
            with output_add_panel:
                clear_output(wait=True)
                new_row = pd.DataFrame({
                    'name':[orphan_name_input.value],
                    'location':[orphan_loc_input.value],
                    'items_needed':[orphan_item_input.value],
                    'urgency':[orphan_urgency_input.value]
                })
                global orphanages
                orphanages = pd.concat([orphanages, new_row], ignore_index=True)
                orphanages.to_csv('orphanages.csv', index=False)
                orphanage_test_input.options = orphanages['name'].tolist()
                print(f"✅ Orphanage '{orphan_name_input.value}' added successfully!")
            refresh_charts()
            refresh_matches()

        def add_donor(b):
            with output_add_panel:
                clear_output(wait=True)
                new_row = pd.DataFrame({
                    'name':[donor_name_input.value],
                    'location':[donor_loc_input.value],
                    'donation_preferences':[donor_pref_input.value],
                    'availability':[donor_avail_input.value]
                })
                global donors
                donors = pd.concat([donors, new_row], ignore_index=True)
                donors.to_csv('donors.csv', index=False)
                print(f"✅ Donor '{donor_name_input.value}' added successfully!")
            refresh_charts()
            refresh_matches()

        add_orphan_button.on_click(add_orphanage)
        add_donor_button.on_click(add_donor)

        display(widgets.HTML("<b>New Orphanage:</b>"))
        display(widgets.VBox([orphan_name_input, orphan_loc_input, orphan_item_input, orphan_urgency_input, add_orphan_button]))
        display(widgets.HTML("<b>New Donor:</b>"))
        display(widgets.VBox([donor_name_input, donor_loc_input, donor_pref_input, donor_avail_input, add_donor_button, output_add_panel]))

        # --- Charts & Matches ---
        display(widgets.HTML("<h3 style='color:#2c3e50'>Donation Insights & Urgency Overview</h3>"))
        display(output_charts)
        refresh_charts()

        display(widgets.HTML("<h3 style='color:#2c3e50'>Matches</h3>"))
        display(output_dashboard)
        update_matches_table()

        # --- Top Donors & Most Urgent Orphanages ---
        display(widgets.HTML("<h3 style='color:#2c3e50'>Top Donors</h3>"))
        top_donors = matches_df['Donor'].value_counts().head(5)
        display(pd.DataFrame({'Donor': top_donors.index, 'Matches': top_donors.values}))

        display(widgets.HTML("<h3 style='color:#2c3e50'>Most Urgent Orphanages</h3>"))
        top_orphanages = orphanages.sort_values(by='urgency', ascending=False).head(5)
        display(top_orphanages[['name','items_needed','urgency']].reset_index(drop=True))

# Run the final dashboard
final_dashboard()
display(output_final_dashboard)
